# Support vector machines for Survival Analysis

In [1]:
import numpy as np
import math
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

## GBSG Data

In [2]:
from google.colab import files
uploaded = files.upload()

Saving GBSG2.csv to GBSG2.csv


In [3]:
GBSG = pd.read_csv('GBSG2.csv', sep = ";")

In [4]:
GBSG.head()

,horTh,age,menostat,tsize,tgrade,pnodes,progrec,estrec,time,cens
0,no,70,Post,21,II,3,48,66,1814,1
1,yes,56,Post,12,II,7,61,77,2018,1
2,yes,58,Post,35,II,9,52,271,712,1
3,yes,59,Post,17,II,4,60,29,1807,1
4,no,73,Post,35,II,1,26,65,772,1


In [5]:
GBSG.dtypes

horTh       object
age          int64
menostat    object
tsize        int64
tgrade      object
pnodes       int64
progrec      int64
estrec       int64
time         int64
cens         int64
dtype: object

In [6]:
d = GBSG.cens
y = GBSG.time

In [7]:
X = GBSG.drop(['time', 'cens'], axis = 1)
c_obj = X.select_dtypes(include='object').columns
X = pd.get_dummies(X, prefix=c_obj, drop_first = True)

In [8]:
X.head()

,age,tsize,pnodes,progrec,estrec,horTh_yes,menostat_Pre,tgrade_II,tgrade_III
0,70,21,3,48,66,0,0,1,0
1,56,12,7,61,77,1,0,1,0
2,58,35,9,52,271,1,0,1,0
3,59,17,4,60,29,1,0,1,0
4,73,35,1,26,65,0,0,1,0


# RankSVM

**Comparable pairs**
A set $P = \{(i, j) | y_i > y_j \cap d_j = 1\}$ containing all comparable pairs. Note p is the number of elements in P.

**Ranking problem**
We need to find the rank function $f(Xi) = (X_iW)$ for the individual i with a constrain: Person have higher value of rank function will have longer survival time.

**Objective loss function with square Hinge-loss**
$$ f(W) = \frac{1}{2}W^TW+\frac{\gamma}{2} \sum_{i,j \in P} max(0, 1 - (X_iW - X_jW))^2$$


**Finding optimal coefficients vector W with sparse matrix A and diagonale matrix D**
$A \in R^{p n}$ with $A_{ki} = 1$ and $A_{kj} = -1$ if $(i,j) \in P$ and 0 ortherwise.

A diagonale matrix $D$ that indiates whether this pair is support vector: $ 1- (X_iW - X_jW) >0$

**The objective loss function** becomes:
$$ f(W) = \frac{1}{2}W^TW+\frac{C}{2} (\mathbb1 - AXW)D(\mathbb1 - AXW)^T$$

In [9]:
n = len(y)
X = np.array(X)
# Construct set P
P = []
for i in range(n):
    for j in range(n):
        if ((d[j] == 1) & (y[i]>y[j])):
            P = np.append(P,[i, j], axis = 0)
P = np.reshape(P, (-1,2)).astype(int)

In [10]:
# Construct matrix A
p = P.shape[0]
A = np.zeros((p, n))
for i in range(p):
    A[i, P[i,0]] = 1
    A[i, P[i,1]] = -1

In [12]:
# Construct maxtrix D
# Initialize W
h = X.shape[1]
W = np.random.rand(h,1)
D = np.concatenate(A@X@W < 1)

#### Training by Newton's method

In [13]:
W = np.random.rand(h,1)
C = 0.0003;
for i in range(5):
    D = np.concatenate(A@X@W < 1)
    Aw = A[D, :]
    pw = np.sum(D==False)
    loss = 1/2 * W.T@W + C/2 *(pw + W.T@X.T@(Aw.T@Aw@X@W - 2*Aw.T@np.ones((p-pw,1))))
    grad = W + C * X.T @ (Aw.T@Aw@X@W - Aw.T@np.ones((p-pw,1)))
    hess = np.eye(h,h) + C * X.T @ Aw.T @ Aw @X
    
    print(loss)
    W = W - np.linalg.inv(hess) @ grad
print('Number of true ranking prediction', np.sum(np.concatenate(A@X@W>0)))

[[93266.5905255]]
[[17.40683619]]
[[-2.37719213]]
[[-2.35856764]]
[[-2.31157278]]
Number of true ranking prediction 92573


# Regression SVM

**Regression problem**
The function $f(Xi) = X_iW+b$ that modelize directly the exact time of an event

**Objective loss function with square Hinge-loss**
$$ f_{reg}(W, b) = \frac{1}{2}W^TW+\frac{\gamma}{2} \sum_{i=1}^n (\zeta_{W,b}(y_i, X_i, d_i) )^2$$

$$\zeta_{W,b}(y_i, X_i, d_i) = \begin{cases} max(0, y_i-X_iW-b) \quad\text{ if } d_i=0\\
y_i-X_iW-b \qquad \qquad \quad\text{if } d_i=1
\end{cases} $$
**Or in matrix form:**
$$ f_{reg}(\omega) = \frac{1}{2}\omega^T\omega+\frac{\gamma}{2} (y-X\omega)^TR_{\omega}(y-X\omega)$$
With $\omega=(b,W)$ and extending $X$ by a column of all ones, $R_{\omega}$ is a diagonale matrix with the i-th element being 1 if $(y_i > X_i\omega)$ or $(d_i=1)$, and zero otherwise.

#### Training by Newton's method

In [ ]:
# Add vector 1 to X
# Initialize W
# Construct matrix R
Xn = [np.ones((n,1), X]
hn = Xn.shape[1]
W = np.random.rand(hn,1)